In [2]:
from queue import PriorityQueue
from enum import Enum

%run ../src/ipython_exit.py

In [3]:
def neighbors(cells, pos):
    neighbor_positions = []
    offsets = [(-1,0),(1,0),(0,-1),(0,1)]
    for (i,j) in offsets:
            if(pos.y+j>=0 and pos.x+i>=0 and
                pos.y+j<nrows and pos.x+i<ncols):
                if(cells[pos.y+j,pos.x+i]!=Label.BODY.value and cells[pos.y+j,pos.x+i]!=Label.HEAD.value):
                    neighbor_positions.append(Position(pos.y+j,pos.x+i))
                
    return neighbor_positions

In [4]:
def manhattan_dist(pos1, pos2):
    return abs(pos1.x - pos2.x) + abs(pos1.y - pos2.y)

In [5]:
def heuristic(pos, other_pos, weights):
    _heuristic = 0
    N = len(weights)
    for i in range(N):
        _heuristic += weights[i] * manhattan_dist(pos, other_pos[i])
    return _heuristic

In [6]:
def reconstruct_path(node_dict, start_pos, goal_pos, PARENT_POS):
    current = goal_pos
    path = []
    while(current!=start_pos):
        try:
            path.append(current)
            current = node_dict[current][PARENT_POS]
        except KeyError:
            print(f"KeyError at position {(current.y, current.x)}")
            exit()
    path.append(start_pos)
    path.reverse()
    
    return path

In [7]:
# node = [f_cost, g_cost, node_pos, parent_node_pos, open, closed]

def astar(cells, start_pos, goal_pos, cm_pos_list, weights):
    F, G, POS, PARENT_POS, OPEN, CLOSED = range(6)
    start_cost = 0
    
    open_pq = PriorityQueue()
    
    start_node = [start_cost, start_cost, start_pos, None, True, False]
    
    open_pq.put(start_node)
    node_dict = {start_pos: start_node}
    
    while(not open_pq.empty()):
        current_node = open_pq.get()
        current_node[OPEN] = False
        current_node[CLOSED] = True
        
        if(current_node[POS]==goal_pos):
            node_dict[goal_pos] = current_node
            break
        
        for neighbor_pos in neighbors(cells, current_node[POS]):
            g_cost = manhattan_dist(neighbor_pos, start_pos)
            
            other_pos = [goal_pos] + cm_pos_list
            h_cost = heuristic(neighbor_pos, other_pos, weights)
            f_cost = g_cost + h_cost
            
            if(not neighbor_pos in node_dict):
                neighbor_node = [f_cost, g_cost, neighbor_pos, current_node[POS], False, False]
                node_dict[neighbor_pos] = neighbor_node
            else:
                neighbor_node = node_dict[neighbor_pos]
            
            if(cells[neighbor_pos.y,neighbor_pos.x]!=Label.BG.value or neighbor_node[CLOSED]):
                continue
            
            if(g_cost < neighbor_node[G] or not neighbor_node[OPEN]):
                neighbor_node[F] = f_cost
                neighbor_node[PARENT_POS] = current_node[POS]
                if(not neighbor_node[OPEN]):
                    open_pq.put(neighbor_node)
                    neighbor_node[OPEN] = True
    
    
    path = reconstruct_path(node_dict, start_pos, goal_pos, PARENT_POS)
    return path